In [2]:
from awpy import Demo
import polars as pl

dem = Demo("mouz-vs-vitality-m3-inferno.dem", verbose=False)
dem.parse(player_props=["health", "armor_value", "yaw", "inventory"])
game_times = dem.parse_ticks(other_props=["game_time", "team_clan_name", 'is_terrorist_timeout', 'is_ct_timeout'])   

In [19]:
# cumulative_wins = (
#     dem.rounds['round_num', 'winner'].with_columns([
#         pl.when(pl.col("winner") == "ct").then(1).otherwise(0).alias("ct_wins"),
#         pl.when(pl.col("winner") == "t").then(1).otherwise(0).alias("t_wins")
#     ])
#     .select([
#         "round_num", 
#         pl.col("ct_wins").cum_sum().alias("ct_wins"),
#         pl.col("t_wins").cum_sum().alias("t_wins")
#     ])
# )

# Create a column for CT and T wins, then apply cumulative sum
df = (
    dem.rounds['round_num', 'winner'].with_columns([
        pl.when(pl.col("winner") == "ct").then(1).otherwise(0).alias("ct_wins"),
        pl.when(pl.col("winner") == "t").then(1).otherwise(0).alias("t_wins")
    ])
    .with_columns([
        pl.col("ct_wins").cum_sum().alias("ct_wins_cumsum"),
        pl.col("t_wins").cum_sum().alias("t_wins_cumsum")
    ])
)

# Shift the cumulative columns by 1 to get "wins during the round"
df = df.with_columns([
    pl.col("ct_wins_cumsum").shift(1).fill_null(0).alias("ct_wins_during_round"),
    pl.col("t_wins_cumsum").shift(1).fill_null(0).alias("t_wins_during_round")
])

# Select relevant columns and display the result
df = df.select([
    "round_num",
    "ct_wins_during_round",
    "t_wins_during_round"
])

print(df.head(10))

shape: (10, 3)
┌───────────┬──────────────────────┬─────────────────────┐
│ round_num ┆ ct_wins_during_round ┆ t_wins_during_round │
│ ---       ┆ ---                  ┆ ---                 │
│ u32       ┆ i32                  ┆ i32                 │
╞═══════════╪══════════════════════╪═════════════════════╡
│ 1         ┆ 0                    ┆ 0                   │
│ 2         ┆ 1                    ┆ 0                   │
│ 3         ┆ 1                    ┆ 1                   │
│ 4         ┆ 1                    ┆ 2                   │
│ 5         ┆ 1                    ┆ 3                   │
│ 6         ┆ 1                    ┆ 4                   │
│ 7         ┆ 1                    ┆ 5                   │
│ 8         ┆ 1                    ┆ 6                   │
│ 9         ┆ 1                    ┆ 7                   │
│ 10        ┆ 1                    ┆ 8                   │
└───────────┴──────────────────────┴─────────────────────┘
